# Analiza: Wykrywanie języka na podstawie częstotliwości słów w tekscie

Celem tej pracy jest ocena, czy na podstawie danych dotyczących częstotliwości występowania słów w arytkule na wybranym wiki (u nas bulbapedia) da się rozpoznać język tekstu. Eksperyment polega na porównaniu listy najczęsciej występujących słów na wiki z listą najczęściej występujących słów w jednym z trzech języków. Sprawdzimy dodatkowo jak na poprawność metody wpływa długość arytkułu.

In [ ]:
# Początkowe importy najważniejszych modułów i bibliotek
import re
import matplotlib.pyplot as plt
from wordfreq import top_n_list, word_frequency
from pathlib import Path
from scraper_logic import Scraper
from bs4 import BeautifulSoup
import requests
import math

Wykorzystamy listy najczęsciej występujących słów z trzech języków: polskiego, angielskiego (język naszej wiki), oraz niemieckiego. Użyjemy do tego biblioteki wordfreq.

In [ ]:
# Zwraca liniową ilość wystąpień słowa 'na jedno słowo' w języku
def get_language_words(lang: str, n: int = 1000) -> dict[str, float]:
    words = top_n_list(lang, n)
    return {w: word_frequency(w, lang) for w in words}

    
langs = ["en", "pl", "de"]

lang_freq = {lang: get_language_words(lang) for lang in langs}

Na potrzeby analizy stworzymy nowy uniwersalny scraper, który zadziała dla każdej strony, nie tylko bulbapedii.

In [36]:
# Nasza funkcja scrapująca zwraca obiekt bs, który potem będziemy edytować zależnie od struktury html
def get_article(url):
    # Tworzymy header, który pozwoli nam się dostać do stron używając headera z httpbin.org/cache
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:147.0) Gecko/20100101 Firefox/147.0'
    }

    try:
        source = requests.get(url, headers=headers)
    except requests.RequestException:
        return None

    if source.status_code != 200:
        print(f"Błąd: status_code={source.status_code} dla {url}")
        return None

    return BeautifulSoup(source.text, 'html.parser')


Jako dłuższy artykuł z wybranej wiki (bulbapedii) posłuży strona z opisami umiejętności pokemonów: [https://bulbapedia.bulbagarden.net/wiki/Ability]. Pobierzmy html tej strony używając naszej nowej funkcji.

In [37]:
lo_soup = get_article('https://bulbapedia.bulbagarden.net/wiki/Ability')

Zrobimy to samo dla krótkiego arytkuły ze strony [https://bulbapedia.bulbagarden.net/wiki/Bulbakaki], który pełen jest nazw własnych.

In [38]:
sh_soup = get_article('https://bulbapedia.bulbagarden.net/wiki/Bulbakaki')

Robimy to również dla artykułów w każdym z wybranych języków:

In [39]:
pl_soup = get_article('https://wolnelektury.pl/katalog/lektura/witkacy-o-czystej-formie.html')
en_soup = get_article('https://minecraft.fandom.com/wiki/Trading')
de_soup = get_article('https://wolnelektury.pl/katalog/lektura/heyse-die-schwarze-jakobe.html')

# Usuwamy polskie słowa z niemieckiego tekstu (bo pobieramy z polskiej strony)
for h2 in de_soup.find_all("h2"):
    h2.decompose()

Zdefiniujmy funkcję do liczenia słów zwracającą słownik:

In [40]:
def count_words(text) -> dict:
    word_dict = {}

    for word in text:
        word_dict[word] = word_dict.get(word, 0) + 1

    return word_dict


Teraz musimy w każdej z tych struktur znaleźć właściwą zawartość z artykułem, zamienić ją na tekst i policzyć słowa:

In [ ]:
# Wycinanie właściwej zawartości (find zwraca typ tag)
sh_content = sh_soup.find('p')
lo_content = lo_soup.find('div', class_='mw-body-content')
pl_content = pl_soup.find('div', class_='main-text-body')
en_content = en_soup.find('div', id='content')
de_content = de_soup.find('div', class_='main-text-body')

contents = {
    "sh": sh_content,
    "lo": lo_content,
    "pl": pl_content,
    "en": en_content,
    "de": de_content,
}

dicts = {}

for name, c in contents.items():
    # Zmieniamy typ na string
    text = c.get_text(' ', strip=True)
    # Tokenizacja
    tokens = re.findall(r'[^\W\d_]+', text.lower())
    # Tworzymy słowniki z licznikiem wystąptień
    res: dict[str, int] = count_words(tokens)
    # Sortujemy
    sorted_dict = dict(sorted(res.items(), key=lambda x: x[1], reverse=True))
    # Wstawiamy do dicts
    dicts[name] = sorted_dict


Upewnijmy się, że wszystko działa wypisując pare pierwszych słów:

In [ ]:
for d in dicts:
    print(list(d.items())[:100])

[('the', 3), ('to', 3), ('bulbagarden', 2), ('oekaki', 2), ('users', 2), ('bulbakaki', 2), ('was', 2), ('known', 1), ('its', 1), ('as', 1), ('an', 1), ('run', 1), ('by', 1), ('which', 1), ('used', 1), ('wacintaki', 1), ('poteto', 1), ('there', 1), ('is', 1), ('a', 1), ('deviantart', 1), ('group', 1), ('for', 1), ('of', 1), ('upload', 1), ('their', 1), ('completed', 1), ('artworks', 1), ('it', 1), ('closed', 1), ('down', 1), ('in', 1)]
[('the', 847), ('s', 540), ('pokémon', 500), ('in', 428), ('a', 286), ('ability', 285), ('to', 280), ('its', 247), ('and', 241), ('of', 237), ('it', 168), ('moves', 139), ('abilities', 132), ('from', 121), ('battle', 118), ('when', 114), ('activated', 113), ('used', 110), ('with', 110), ('is', 108), ('by', 93), ('iii', 91), ('that', 87), ('type', 80), ('stat', 70), ('was', 68), ('damage', 63), ('as', 62), ('have', 61), ('hidden', 58), ('or', 56), ('iv', 55), ('attack', 54), ('vii', 53), ('an', 52), ('v', 52), ('ix', 50), ('power', 49), ('viii', 48), ('be'

Pora zdefiniować funckje szacującą język tekstu. Pomysł: liczymy współczynnik **COVERAGE** czyli jaka część wszystkich słów w tekście należy do top-N słów danego języka oraz **COSINE SIMILARITY** czyli porównujemy proporcje tych top-słów w tekście do proporcji w profilu języka (wpisujemy słowa w wektory i patrzymy na kąt pomiędzy nimi mówiący czy idą w tą samą stronę). Wynikiem końcowym będzie coverage * cosine. Zacznijmy od funkcji porównującej proporcje, wzór to:
$$
\cos(\theta)=\frac{x\cdot y}{\|x\|\|y\|}
$$
Gdzie $0^\circ$ oznacza całkowite podobieństwo, a $90^\circ$ jego brak. nie otrzymamy ujemych wartości bo wszystkie el. w wektorach są dodatnie.

Nasza norma wektora to:
$$
\|x\| = \sqrt{\sum_{i=1}^{n} x_i^2}
$$


In [2]:
def cosine_similarity(vec_a: dict[str, float], vec_b: dict[str, float]) -> float:
    # Ustalamy wspólną liste słów dla obu wektorów
    keys = set(vec_a.keys()) | set(vec_b.keys())

    # Iloczyn skalarny wektorów i ich normy
    dot = 0.0 
    norm_a = 0.0
    norm_b = 0.0

    # Iterujemy po każdym słowie i uzupełniamy dane
    for k in keys:
        a = vec_a.get(k, 0.0)
        b = vec_a.get(k, 0.0)

        dot += a * b
        norm_a += a * a
        norm_b += b * b

    # Obliczamy normy
    norm_a = math.sqrt(norm_a)
    norm_b = math.sqrt(norm_b)

    # Dla wektorów pustych zwróć 0 dopasowania
    if norm_a == 0.0 or norm_b == 0.0:
        return 0.0
    
    # Zwróc współczynnik cos pomiędzy 0 a 90 stopni
    return dot / (norm_a * norm_b)


Zdefiniujmy funkcję, która na bazie coverage i cosine similarity obliczy współczynnik dopasowania języka, zwracającą wynik 0..1 gdzie 0 to brak dopasowania a 1 to pewne dopasowanie.

In [ ]:
def lang_confidence_score(word_counts: dict[str, int],
                          language_words_with_frequency: dict[str, float]) -> float:
    if not word_counts or not language_words_with_frequency:
        return 0.0
    
    total_words = sum(word_counts.values())
    if total_words <= 0:
        return 0.0
    
    # Budujemy wektory na podstawie słowników częstotliwości
    lang_weights: dict[str, float] = {}

    for w, f in language_words_with_frequency.items():
        weight = float(f)
        if weight > 0:
            lang_weights[w] = weight
    
    if not lang_weights:
        return 0.0

    